In [ ]:
!pip install -q matplotlib


In [ ]:
import matplotlib.pyplot as plt 
import tensorflow as tf
import os
from tensorflow.keras import activations
import numpy as np
import tensorflow_datasets as tfds
from tensorflow.keras import regularizers
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# import cv2

print("Num GPUs Available: ", tf.test.is_gpu_available())


In [ ]:
def plot_history_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
def plot_history_accuracy(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
dogvscat = "./train/"
IMG_SIZE=128
batch_size = 30
EAPOCHS = 20
input = tf.keras.Input(shape = ((IMG_SIZE,IMG_SIZE,3)))
train_dir=os.path.join('./', 'train/')

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE),)
train_ds = train_ds.unbatch()
train_xs = []
train_ys = []
for x,y in train_ds:
    train_xs.append(x.astype('float32')/255.)
    train_ys.append(y)
train_xs = tf.convert_to_tensor(train_xs)
train_ys = tf.convert_to_tensor(train_ys)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE),)
val_ds = val_ds.unbatch()
test_xs = []
test_ys = []
for x,y in val_ds:
    test_xs.append(x.astype('float32')/255.)
    test_ys.append(y)
test_xs = tf.convert_to_tensor(test_xs)
test_ys = tf.convert_to_tensor(test_ys)

In [ ]:
encoder_input = input
x = tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same')(encoder_input)
x = tf.keras.layers.MaxPooling2D((2,2),padding='same')(x)
x = tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2,2),padding='same')(x)
x = tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2,2),padding='same')(x)
x= tf.keras.layers.Flatten()(x)
encoder_output = tf.keras.layers.Dense(512, activation='relu', name="ksks")(x)


decoder_input = tf.keras.layers.Dense(8192, activation='relu')(encoder_output)
x = tf.keras.layers.Reshape((16,16,32))(decoder_input)
x = tf.keras.layers.Conv2DTranspose(32, (3,3), strides=2, activation='relu' ,padding='same')(x)
x = tf.keras.layers.Conv2DTranspose(64, (3,3), strides=2, activation='relu' ,padding='same')(x)
x = tf.keras.layers.Conv2DTranspose(64, (3,3), strides=2, activation='relu' ,padding='same')(x)
decoder_output = tf.keras.layers.Conv2D(3,(3,3),activation='relu' ,padding='same')(x)


autoencoder = tf.keras.Model(inputs = encoder_input,outputs = decoder_output, name='ae')
autoencoder.summary()
opt = tf.keras.optimizers.Adam(learning_rate= 0.001,decay=1e-12)
autoencoder.compile(loss = 'mse', optimizer=opt)

In [ ]:
history_autoencoder = autoencoder.fit(
    train_xs,
    train_xs,
    batch_size = 30,
    validation_split=0.1,
    epochs=EAPOCHS*5,
  )
autoencoder.save(f'autoencoder_for_cnn_128.h5')
encoder_model = tf.keras.Model(encoder_input,encoder_output,name='encoder')
encoder_model.save(f'encoder_for_cnn_128.h5')

CNN with encoder from Autoencoder

In [ ]:
encoder_model = tf.keras.models.load_model('encoder_for_cnn_128.h5')
cnn_with_encoder_output = tf.keras.layers.Dense(1, activation='sigmoid')(encoder_output)
cnn_with_encoder_model = tf.keras.Model(inputs = encoder_input,outputs = cnn_with_encoder_output, name='cnn_encoder')
cnn_with_encoder_model.summary()
opt = tf.keras.optimizers.Adam(learning_rate= 0.0001,decay=1e-6)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


In [ ]:
encoder_model = tf.keras.Model(encoder_input,encoder_output,name='encoder')
autoencoder_for_cnn =  tf.keras.models.load_model('autoencoder_for_cnn_128.h5')
for l1,l2 in zip(cnn_with_encoder_model.layers[:len(encoder_model.layers)],autoencoder_for_cnn.layers[0:len(encoder_model.layers)]):
    l1.set_weights(l2.get_weights())

In [ ]:
cnn_with_encoder_model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
history_cnn_with_encoder = cnn_with_encoder_model.fit(
    train_xs,
    train_ys,
    batch_size = 30,
    validation_split=0.1,
    epochs=EAPOCHS*5,
    callbacks=[callback]
  )
cnn_with_encoder_model.save(f'cnn_with_encoder_model_128.h5')

In [ ]:
plot_history_loss(history_autoencoder)
plot_history_loss(history_cnn_with_encoder)
plot_history_accuracy(history_cnn_with_encoder)
print(cnn_with_encoder_model.evaluate(test_xs,test_ys))

Normal CNN

In [ ]:
cnn_input = input
x = tf.keras.layers.Conv2D(32, (4,4), activation='relu',padding='same')(cnn_input)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.MaxPooling2D((2,2),padding='same')(x)
x = tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding='same')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.MaxPooling2D((2,2),padding='same')(x)
x = tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.MaxPooling2D((2,2),padding='same')(x)
cnn_Amos = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(cnn_Amos)
x = tf.keras.layers.Dropout(0.1)(x)
cnn_output =   tf.keras.layers.Dense(1, activation='sigmoid')(x)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
cnn_model = tf.keras.Model(inputs = cnn_input,outputs = cnn_output, name='cnn')
cnn_model.summary()
opt = tf.keras.optimizers.Adam(learning_rate= 0.0001,decay=1e-9)
# opt = tf.keras.optimizers.RMSprop(learning_rate= 0.001,decay=1e-6)
cnn_model.compile(loss = tf.losses.BinaryCrossentropy(), optimizer=opt,metrics=['accuracy'])

In [ ]:
history_cnn = cnn_model.fit(
    train_xs,
    train_ys,
    batch_size = 30,
    validation_split=0.1,
    epochs=int(EAPOCHS*12),
    callbacks=[callback]
  )
# cnn_model.save(f'cnn_model_128_2048.h5')

In [ ]:
plot_history_loss(history_cnn)
plot_history_accuracy(history_cnn)
print(cnn_model.evaluate(test_xs,test_ys))

In [ ]:
model = cnn_with_encoder_model
import matplotlib.pyplot as plt
img = tf.keras.utils.load_img(dogvscat+'dog/dog.2.jpg', target_size=(IMG_SIZE,IMG_SIZE))
#img=tf.zeros([100,100,3])
layer=tf.keras.Model(inputs=model.input,outputs=model.layers[2].output)
# img=tf.random.uniform((IMG_SIZE,IMG_SIZE,3))
img=tf.keras.utils.img_to_array(img)
img=tf.expand_dims(img, axis=0)
feature_map=layer.predict(img)
square = 10
ix = 1
fig=plt.figure(figsize=(30,30))
k=2
for _ in range(square):
    for _ in range(square):
        # specify subplot and turn of axis
        if sum(sum(feature_map[0, :, :, ix-1]))>0.7:
            ax = fig.add_subplot(square, square, k-1)
            ax.set_xticks([])
            ax.set_yticks([])
        # plot filter channel in grayscale
            plt.imshow(feature_map[0, :, :, ix-1])
            k+=1
        ix += 1
# show the figure
plt.show()